In [60]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter
from mpl_toolkits import mplot3d
from PIL import Image, ImageDraw
from matplotlib.backends.backend_agg import FigureCanvasAgg
import numpy as np
import matplotlib
matplotlib.use('Agg')

In [287]:
class ACO:
    def __init__(self, num_pop, graph):
        n = len(graph)

        self.graph = graph

        self.population = [[0]] * num_pop # Each ant starts at city number 0
        self.dist = np.zeros(shape=(num_pop)) # dist[ant] = distance made so far

        self.pheromone = np.ones((n, n)) # Ones, so that alpha > 1 will increase probability 

        self.adjacency_matrix = np.zeros(shape=(n, n), dtype=np.float32) # [i, j] = Euclidean distance between city i and j

        # Setting up the distances
        for i in range(n):
            self.adjacency_matrix[i] = np.apply_along_axis(np.linalg.norm, 1, graph[i] - graph)

        # Banning paths from city to itself 
        self.adjacency_matrix = np.where(self.adjacency_matrix == 0, np.inf, self.adjacency_matrix)

    def main_loop(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromone) # To perform pheromon decay after loop ends
            
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromone[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Setting visited cities probabilities to zero
                    for j in range(n):
                        if j in self.population[ant]:
                            tmp[j] = 0

                    # Probabilities itself
                    prob = tmp / tmp.sum()

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())

                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromone[next_city][current_city] += Q / self.dist[ant]
                    self.pheromone[current_city][next_city] += Q / self.dist[ant]

                    delta_tau[next_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                    if len(self.population[ant]) == n:
                        # Connecting the last city with the first
                        self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                # There's some bug which I don't know how to fix
                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromone = (1 - rho) * self.pheromone + delta_tau
        
        return min_dist, best_route
    
    def main_loop_fast(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix
        ants = np.array(range(len(self.population)))

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromone) # To perform pheromon decay after loop ends
            
            self.fast_ant(self, ants, alpha=alpha, mu=mu, beta=beta, n=n, Q=Q, delta_tau=delta_tau)

            min_ant = np.argmin(self.dist[self.dist != 0])

            if min_dist > self.dist[min_ant] and self.dist[min_ant] != 0:
                min_dist = np.copy(self.dist[min_ant])
                best_route = np.copy(self.population[min_ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromone = (1 - rho) * self.pheromone + delta_tau
        
        return min_dist, best_route
    

    @np.vectorize(excluded=['self','alpha', 'mu', 'beta', 'n', 'Q', 'delta_tau'])
    def fast_ant(self, ant, alpha, mu, beta, n, Q, delta_tau):
        while len(self.population[ant]) != n:
            # Getting the last city that ant have visited so far
            current_city = self.population[ant][-1]
            # Temporary variable to compute probabilities later
            tmp = (self.pheromone[current_city] ** alpha) * (mu[current_city] ** beta)

            # Setting visited cities probabilities to zero
            for j in range(n):
                if j in self.population[ant]:
                    tmp[j] = 0

            # Probabilities itself
            prob = tmp / tmp.sum()

            # Cumulative probabilites to choose next city
            cum_prob = np.cumsum(prob)

            # Choosing next city while it hasn't been visited
            next_city = np.searchsorted(cum_prob, np.random.rand())

            while next_city in self.population[ant]:
                next_city = np.searchsorted(cum_prob, np.random.rand())

            self.population[ant].append(next_city)
            self.dist[ant] += self.adjacency_matrix[current_city][next_city]

            # Updating pheromon
            self.pheromone[next_city][current_city] += Q / self.dist[ant]
            self.pheromone[current_city][next_city] += Q / self.dist[ant]

            delta_tau[next_city][next_city] += Q / self.dist[ant]
            delta_tau[current_city][next_city] += Q / self.dist[ant]

            if len(self.population[ant]) == n:
                # Connecting the last city with the first
                self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]
    

    def main_loop_plot(self, num_iter, alpha, beta, Q, rho, gif_name="ACO"):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        # plotting
        writer = PillowWriter(fps=2, duration=0.5)
        fig = plt.figure()

        with writer.saving(fig, f"{gif_name}.gif", 100):
            for i in range(num_iter):
                self.population = [[0]] * len(self.population) # Each ant starts at city number 0
                self.dist = np.zeros(self.dist.shape)
                delta_tau = np.zeros_like(self.pheromone) # To perform pheromon decay after loop ends
                
                for ant in range(len(self.population)):
                    # Looping until ant hasn't visited all of the cities
                    while len(self.population[ant]) != n:
                        # Getting the last city that ant have visited so far
                        current_city = self.population[ant][-1]

                        # Temporary variable to compute probabilities later
                        tmp = (self.pheromone[current_city] ** alpha) * (mu[current_city] ** beta)

                        # Setting visited cities probabilities to zero
                        for j in range(n):
                            if j in self.population[ant]:
                                tmp[j] = 0

                        # Probabilities itself
                        prob = tmp / tmp.sum()

                        # Cumulative probabilites to choose next city
                        cum_prob = np.cumsum(prob)

                        # Choosing next city while it hasn't been visited
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                        while next_city in self.population[ant]:
                            next_city = np.searchsorted(cum_prob, np.random.rand())

                        self.population[ant].append(next_city)
                        self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                        # Updating pheromon
                        self.pheromone[next_city][current_city] += Q / self.dist[ant]
                        self.pheromone[current_city][next_city] += Q / self.dist[ant]

                        delta_tau[next_city][next_city] += Q / self.dist[ant]
                        delta_tau[current_city][next_city] += Q / self.dist[ant]

                        if len(self.population[ant]) == n:
                            # Connecting the last city with the first
                            self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                    # There's some bug which I don't know how to fix
                    if min_dist > self.dist[ant] and self.dist[ant] != 0:
                        min_dist = np.copy(self.dist[ant])
                        best_route = np.copy(self.population[ant])

                    arr_to_plot = self.graph[best_route]

                    plt.plot(arr_to_plot[:, 0], arr_to_plot[:, 1])
                    plt.scatter(self.graph[:, 0], self.graph[:, 1], color="red")

                    writer.grab_frame()

                    plt.cla()

                print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

                # Pheromone decay
                self.pheromone = (1 - rho) * self.pheromone + delta_tau
        
        return min_dist, best_route
    

    def main_loop_plot_2(self, num_iter, alpha, beta, Q, rho, gif_name="ACO"):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        # plotting
        fig, ax = plt.subplots()
        images = []
        canvas = FigureCanvasAgg(fig)

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromone) # To perform pheromon decay after loop ends
            
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromone[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Setting visited cities probabilities to zero
                    for j in range(n):
                        if j in self.population[ant]:
                            tmp[j] = 0

                    # Probabilities itself
                    prob = tmp / tmp.sum()

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())

                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromone[next_city][current_city] += Q / self.dist[ant]
                    self.pheromone[current_city][next_city] += Q / self.dist[ant]

                    delta_tau[next_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                    if len(self.population[ant]) == n:
                        # Connecting the last city with the first
                        self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                # There's some bug which I don't know how to fix
                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

                arr_to_plot = self.graph[best_route]

                ax.plot(arr_to_plot[:, 0], arr_to_plot[:, 1])
                ax.scatter(self.graph[:, 0], self.graph[:, 1], color="red")

                # Render the plot as an RGBA buffer
                canvas.draw()
                buf = canvas.buffer_rgba()

                # Create a PIL Image from the RGBA buffer
                image = Image.frombytes('RGBA', canvas.get_width_height(), buf, 'raw', 'RGBA')
                images.append(image)
                plt.cla()

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromone = (1 - rho) * self.pheromone + delta_tau
    

        images[0].save(f'{gif_name}.gif',
               save_all=True, append_images=images[1:], optimize=False, duration=10, loop=0)
        return min_dist, best_route

In [288]:
def create_circle(n, radius=1):
    circle = []
    theta = 0
    delta = 2 * np.pi / n

    side_length = 2 * radius * np.sin(delta / 2)

    for _ in range(n):
        x = np.cos(theta)
        y = np.sin(theta)
        circle.append(np.array([x, y]))
        theta += delta


    return np.array(circle), n * side_length

In [289]:
file_path = r'D:\Opera_Download\xqf131.tsp'
def read_tsp_file(filepath):
    tsp = []
    with open(filepath) as file:
        for _ in range(8):
            file.readline()
        
        for line in file.readlines()[:-1]:
            _, x, y = line.split()
            tsp.append(np.array([np.float32(x), np.float32(y)]))

    return np.array(tsp)

In [295]:
"""
Min distance: 6.868
Min route [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 17 19]
Real ans 6.257

Min route [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Min dist: 6.272
 Real min dist: 6.272

 (num_iter=2_000, alpha=1, beta=0.5, rho=0, Q=6)
"""
# tsp1 = read_tsp_file(file_path) # Q = 564
graph, perim = create_circle(50)
test = ACO(200, graph)
dist, route = test.main_loop_fast(num_iter=5000, alpha=1, beta=0.5, rho=0, Q=6)
print("Min dist: %.3f\n Real min dist: %.3f" % (dist, perim))

Iteration: 1
Min distance: 49.700
Min route [ 0 43 38  5  6 49 27 31 26 28 32 30 18 17 12 48 45  8 34 33 37 41  7 42
 46  3  4 24 29 35 14 11 23 47  1 44 22 16 25 20 13 15 19  2 40 36  9 39
 21 10]
Iteration: 2
Min distance: 49.372
Min route [ 0 43  7  8 41  5 39 15 31 13 10  3  4 44 38 20 33 37 35 36 27 42  1  2
 14 11 18 19 28 32 30 22 26 23 46 48 49 24 34 25 40 47 17 16 12 21  9  6
 29 45]
Iteration: 3
Min distance: 49.372
Min route [ 0 43  7  8 41  5 39 15 31 13 10  3  4 44 38 20 33 37 35 36 27 42  1  2
 14 11 18 19 28 32 30 22 26 23 46 48 49 24 34 25 40 47 17 16 12 21  9  6
 29 45]
Iteration: 4
Min distance: 49.372
Min route [ 0 43  7  8 41  5 39 15 31 13 10  3  4 44 38 20 33 37 35 36 27 42  1  2
 14 11 18 19 28 32 30 22 26 23 46 48 49 24 34 25 40 47 17 16 12 21  9  6
 29 45]
Iteration: 5
Min distance: 47.274
Min route [ 0  5  6  3  4 24 40 41 48 47 49  2 19 28 27 33 21 31 30 18 29 22 20 34
 10  9 12 15 14 38 36 35 26  8 42 16 43 37 23 44  1 17  7 11 13 39 25 32
 45 46]
Iteration:

In [261]:
@np.vectorize(excluded=['l'])
def foo(x, l):
    x **= 2
    l -= x

l = np.array([100, 234, 123])
foo([2, 3, 4, 5], l=l)
l

array([ 42, 176,  65])